In [ ]:
import requests
import csv
import re
import json
import urllib.request
from bs4 import BeautifulSoup
import os
import glob
import pandas as pd

In [ ]:
vano = 2022
for vmes in range(5, 13):
    ano=str(vano)
    mes=""
    if vmes<10 :
        mes="0"         
    mes+=str(vmes)
    mesano=mes+ano
    anomes=ano+mes
    aux ="https://sisgvarmazenamento.blob.core.windows.net/prd/PublicacaoPortal/Arquivos/"
    aux+=anomes       
    aux+=".htm"
    #Consulte o site e retorne o html para a variável 'page'
    page = urllib.request.urlopen(aux)
    soup = BeautifulSoup(page, 'html.parser',from_encoding='utf-8')

    # Criar um arquivo para gravar, adicionar linha de cabeçalhos
    f = csv.writer(open(('Dados'+mesano+'.csv'), 'w'))
    f.writerow(['Vereador','Tipo_de_Gasto','Nome_Da_Empresa','CNPJ','Valor','Mes/Ano'])
    jsonFilePath=('Dados'+mesano+'.json')
    jsondata = [] 
    
    #Regex para remover tags html
    TAG_RE = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    def remove_tags(text):
        return TAG_RE.sub('', text)

    #Inicialização de variaveis
    index=0
    start=0
    nomeVereador=""
    categoriaDespesa=""
    cnpj=""
    LugarDespesa=""
    skip=0
    ignore=0
    bugduplo=1
    #Percorre buscando Tds
    name_list = soup.find('body')
    name_list2 = soup.find(class_="bloco")
    for tr in name_list.find_all('tr'):
        if bugduplo==0:
            bugduplo=-1
            continue
        if(tr.find(text=re.compile("[\s\S]+(Vereador)\\((a)\\)[:\s]",re.I))):
            start=1
            name_list_itemsp = tr.find_all('td')
            for name in name_list_itemsp:
                if(name.find(text=re.compile("[\s\S]+(Vereador)\\((a)\\)",re.I))):
                    names = str(name.contents[1])
                    names=re.sub("[\s\S]+(Vereador)\\((a)\\)[:\s]","",names)
                    names=remove_tags(names)
                    nomeVereador=names
                    ignore=0
                    if bugduplo==1:
                        bugduplo=0
        if(start!=0):
            name_list_itemsv = tr.find_all('td')
            for name in name_list_itemsv:                   
                names = str(name.contents[0])
                if(skip==1):
                    skip=0
                    continue
                if(ignore==1):
                    continue
                names=remove_tags(names)
                names=re.sub("(Natureza da despesa)","",names)
                names=re.sub("(Valor utilizado)","",names)                    
                names=re.sub("(VALORES GASTOS)","",names)
                names=re.sub("(VALORES DISPONIBILIZADOS)","",names)
                names=re.sub("(TOTAL DO ITEM)","VXASkip",names)
                names=re.sub("(TOTAL DO MÊS)","VXBSkip",names)
                names=re.sub("(VEREADOR AFASTADO)","VXBSkip",names)
                if re.match("\d{2}.?\d{3}.?\d{3}/?\d{4}-?\d{2}", names) is not None:              #RegexCNPJ
                    start=2
                if re.match("[\s\S]*(VXASkip)", names) is not None:              #Corte Aqui  
                    start=1;
                    skip=1;
                    continue 
                if re.match("[\s\S]*(VXBSkip)", names) is not None:              #Corte Aqui  
                    start=0;
                    ignore=1
                    break 
                if re.match(r'^\s*$', names):
                    continue
                if(start==1):
                    categoriaDespesa=names;
                    start=2;
                    continue            
                if(start==2):   
                    cnpj=names;
                    start=3
                    continue
                if(start==3):   
                    LugarDespesa=names;
                    start=4
                    continue    
                    #f = csv.writer(open(('Dados'+nomeVereador+'.csv'), 'w'))
                    #f.writerow(['Vereador','Tipo_de_Gasto','Nome_Da_Empresa','CNPJ','Valor','Mes/Ano'])
                print(nomeVereador+", "+mes+"/"+ano)
                #print(nomeVereador+", "+categoriaDespesa+", "+LugarDespesa+", "+cnpj+", "+names+", "+mes+"/"+ano)
                if(start>=4):
                    #Codigo para vereador 
                    try:
                        t = open(('Dados'+nomeVereador+'.csv'), 'a')
                        g = csv.writer(t)
                    except:
                            t = open(('Dados'+nomeVereador+'.csv'), 'w')
                            g = csv.writer(t)
                    g.writerow(['Vereador','Tipo_de_Gasto','Nome_Da_Empresa','CNPJ','Valor','Mes/Ano'])
                    f.writerow([nomeVereador,categoriaDespesa,LugarDespesa,cnpj,names,(mes+"/"+ano)])    
                    g.writerow([nomeVereador,categoriaDespesa,LugarDespesa,cnpj,names,(mes+"/"+ano)]) 
                    t.close()
                    content={}
                    content['Vereador'] = nomeVereador
                    content['Tipo_de_Gasto'] = categoriaDespesa
                    content['Nome_Da_Empresa'] = LugarDespesa
                    content['CNPJ'] = cnpj
                    content['Valor'] = names
                    content['Mes/Ano'] = (mes+"/"+ano)
                    jsondata.append(content)
                    start=2;
                start+=1
    #Ainda dentro do for de mes
    with open(jsonFilePath, 'w') as outfile:
        json.dump(jsondata, outfile)

In [ ]:
# Rode esse comando para saber onde esta o arquivo python
os.getcwd()

In [ ]:
# ATENÇÃO
# Use este recurso se precisar alterar a pasta que o notebook está rodando
os.chdir('d:\\Users\\Rafael\\Documents\\OSB\\4-EXTRACAO DE DADOS\\SCRIPTS')

## 2 - Abrindo arquivos e consolidadndo em somente UM

Agora nesta etapa iremos abrir todos os CSV resultantes da coleta e iremos concatenar todos eles em um arquivo único CSV!

In [ ]:
# Abrindo todos os CSV
tipo_arquivo = 'csv'
todos_csv = [i for i in glob.glob('*.{}'.format(tipo_arquivo))]

In [ ]:
# Combinando todos arquivos csv em um DataFrame
combinado_csv = pd.concat([pd.read_csv(f, encoding='latin-1') for f in todos_csv ])

In [ ]:
# Exportando o DF para um arquivo CSV. É importante fazer isso direto para evitar erros nas etapas seguintes
combinado_csv.to_csv( "gastos_vereadores_2022_05_a_12.csv", index=False, encoding='latin-1')

## 3 - Abrindo o arquivo CSV consolidado para fazer tratamento final

Nesta etapa iremos remover as linhas em branco entre os dados!

In [ ]:
df_vereadores = pd.read_csv('gastos_vereadores_2022_05_a_12.csv', encoding='latin-1')

In [ ]:
df_vereadores.tail()

## 4 - Limpar a tabela

In [ ]:
# Usando maneira mais sofisticada : List Comprehension
indices=[x for x in df_vereadores.loc[df_vereadores['Vereador'] == 'Vereador'].index ]
df_vereadores_final = df_vereadores.drop(indices)


# Ou podemos fazer assim também
'''
df_remove = df_adilson.loc[ df_adilson['Vereador'] == 'Vereador']  
df_final_adilson = df_adilson.drop(df_remove.index)
'''

In [ ]:
# Verificando se processo removeu as linhas em branco
df_vereadores_final.shape

Para garantir que está tudo certo, vamos checar o último vereador na tabela (Fernando Holiday).  
Fiz um slice e verifiquei que os dados do site estão congruentes com os dados obtidos aqui!  
Veja abaixo os dados e compare com o site!

In [ ]:
# Fazendo uma validação para confirmar

df_vereadores_final_ver = df_vereadores_final[df_vereadores_final['Vereador'].str.contains('ELAINE DO QUILOMBO PERIFÉRICO')]
df_vereadores_final_ver.tail(7)

In [ ]:
df_vereadores_final_ver.info()

# Exportando os dados para Excel

In [ ]:
# salvando o DF para um arquivo excel
df_vereadores_final.to_excel( "gastos_vereadores_2022_05_a_12.xlsx", index=False, encoding='utf-8-sig')

# Fontes:


https://www.freecodecamp.org/news/how-to-combine-multiple-csv-files-with-8-lines-of-code-265183e0854/  
https://stackoverflow.com/questions/20906474/import-multiple-csv-files-into-pandas-and-concatenate-into-one-dataframe  
https://pt.stackoverflow.com/questions/385478/remover-linhas-de-um-dataframe-que-atendem-a-certa-condi%C3%A7%C3%A3o

